# Gibson Assembly Primer Design - Demo Notebook

This notebook demonstrates how to use the `gibson_primer_design` module for **binary Gibson assembly** - where each step adds one insert to a vector.

## Features:
- Automatic primer design with optimal Tm calculation
- 4 primers per step (2 for insert, 2 for vector)
- PCR simulation using pydna
- Assembly verification
- Multi-step sequential assembly planning
- Customizable homology regions

In [2]:
    # Install required packages if needed (run in a Jupyter cell)
    %pip install pydna biopython

Note: you may need to restart the kernel to use updated packages.


In [3]:
from gibson_primer_design import GibsonPrimerDesigner
from pydna.dseqrecord import Dseqrecord
from pprint import pprint

## Example 1: Single Step Assembly

Design primers to insert a gene into a plasmid at a specific position.

In [4]:
# Create a circular vector (plasmid backbone)
vector_seq = "ATGCGTACGTTAGCCTAGGCTAGCTAGGCTTACGATGCGTACGTTAGCCTAGGCTAGCTAGGCTTACG" * 10
vector = Dseqrecord(vector_seq, circular=True, name="pVector")

print(f"Vector length: {len(vector)} bp")
print(f"Vector is circular: {vector.circular}")

Vector length: 680 bp
Vector is circular: True


In [5]:
# Create an insert sequence
insert_seq = "GGGAAAATTTCCCGGGTTTAAACCCGGGAAAATTTCCCGGGTTTAAA"
insert = Dseqrecord(insert_seq, name="GeneOfInterest")

print(f"Insert length: {len(insert)} bp")

Insert length: 47 bp


In [6]:
# Initialize the primer designer
# You can customize:
# - homology_length: overlap for Gibson assembly (default 40 bp)
# - target_tm: target melting temperature for primers (default 60°C)
# - min_anneal_length: minimum annealing region (default 20 bp)

designer = GibsonPrimerDesigner(
    homology_length=40,      # 40 bp overlap for Gibson
    min_anneal_length=20,    # At least 20 bp annealing
    target_tm=60.0,          # Target Tm of 60°C
    tm_tolerance=5.0         # Accept ±5°C deviation
)

In [ ]:
# Design primers for inserting at position 100
insert_site = 100  # Where in the vector to insert

result = designer.design_primers(vector, insert, insert_site)

# Print comprehensive summary
designer.print_primer_summary(result)

## Access Individual Primer Sequences

You can easily access the primer sequences for ordering:

In [ ]:
print("\nPRIMERS FOR ORDERING:")
print("=" * 80)
for name, seq in result['primers'].items():
    print(f"{name}: {seq}")

## Example 2: Multi-Step Sequential Assembly

Plan a multi-step assembly where you add multiple inserts one by one.

In [ ]:
# Create multiple inserts
insert1 = Dseqrecord("GGGAAAATTTCCCGGGTTTAAACCCGGG", name="Insert1")
insert2 = Dseqrecord("TTTGGGCCCAAAATTTGGGCCCAAATTT", name="Insert2")
insert3 = Dseqrecord("AAACCCTTTGGGAAATTTCCCGGGTAAA", name="Insert3")

inserts = [insert1, insert2, insert3]

In [ ]:
# Plan all steps
steps = designer.plan_multi_step_assembly(
    initial_vector=vector,
    inserts=inserts,
    insert_site=100  # All inserts go to same position (sequentially)
)

In [ ]:
# Print primers for each step
for step in steps:
    print(f"\n\n{'#'*80}")
    print(f"STEP {step['step']} PRIMERS")
    print(f"{'#'*80}")
    designer.print_primer_summary(step)

## Example 3: Working with Real Sequences

You can load sequences from files (GenBank, FASTA, etc.) using pydna:

In [ ]:
# Example: Load from GenBank file
# from pydna.readers import read
# 
# vector = read("plasmid.gb")[0]
# insert = read("gene.gb")[0]
# 
# result = designer.design_primers(vector, insert, insert_site=500)
# designer.print_primer_summary(result)

## Example 4: Detailed Primer Information

Access detailed information about each primer:

In [ ]:
# Get detailed info for insert forward primer
insert_fwd_info = result['primer_details']['insert_fwd']

print("Insert Forward Primer Details:")
print(f"  Full sequence: {insert_fwd_info['sequence']}")
print(f"  Total length: {insert_fwd_info['length']} bp")
print(f"  Homology tail: {insert_fwd_info['homology_tail'][:20]}... ({insert_fwd_info['homology_length']} bp)")
print(f"  Annealing region: {insert_fwd_info['anneal_seq']} ({insert_fwd_info['anneal_length']} bp)")
print(f"  Annealing Tm: {insert_fwd_info['anneal_tm']}°C")
print(f"  Full primer Tm: {insert_fwd_info['full_tm']}°C")

## Example 5: Verify Assembly

The module automatically simulates PCR and assembly to verify the design:

In [ ]:
# Check PCR products
insert_pcr = result['pcr_products']['insert']
vector_pcr = result['pcr_products']['vector']

print(f"Insert PCR product: {len(insert_pcr)} bp")
print(f"Vector PCR product: {len(vector_pcr)} bp")

# Check assembly result
if result['assembly_result']:
    assembly = result['assembly_result']
    print(f"\n✓ Assembly successful!")
    print(f"  Final plasmid size: {len(assembly)} bp")
    print(f"  Circular: {assembly.circular}")
else:
    print("\n✗ Assembly failed - check parameters")

## Example 6: Custom Homology Regions

You can customize where the homology falls by adjusting the insert_site parameter:

In [ ]:
# Design with different insertion sites to control homology placement
results_different_sites = []

for site in [50, 100, 200, 300]:
    r = designer.design_primers(vector, insert, insert_site=site)
    results_different_sites.append((site, r))
    print(f"\nInsertion at site {site}:")
    print(f"  Vector left homology starts at: {site - designer.homology_length}")
    print(f"  Vector right homology starts at: {site}")

## Tips for Using This Module:

1. **Homology Length**: 40 bp is optimal for Gibson assembly. Can use 20-80 bp.
2. **Insert Site**: Choose a unique restriction site or specific position in your vector.
3. **Tm Values**: Primers are designed for similar Tm (~60°C) for efficient PCR.
4. **Circular Vectors**: The module handles circular DNA properly for plasmids.
5. **Multi-Step**: Each step updates the vector, so subsequent inserts go into the modified plasmid.

## Primer Design Logic:

For **binary Gibson assembly**, each step requires:
- **Insert primers**: Add vector homology so insert PCR product has ends that match vector
- **Vector primers**: Add insert homology so vector PCR product has ends that match insert
- After PCR, the two products have complementary ends and will anneal during Gibson assembly